#### Dependecies

scipy = 0.17.0
gensim = 1.0.1
pandas = 0.19.2
numpy = 1.11.2

#### I used the following code in Pyspark to generate and save on disk the csv file below

select_noteevents2 = spark.sql("""
SELECT text
FROM noteevents2
""")

select_noteevents2.coalesce(1).write.csv("/Volumes/EXTERNAL1/MIMICIII/select_noteevents2-v3", header=True)

select_noteevents3 = spark.sql("""
SELECT subject_id, hadm_id
FROM noteevents2
""")

select_noteevents3.coalesce(1).write.csv("/Volumes/EXTERNAL1/MIMICIII/select_noteevents2-v4", header=True)

#### then I proceed to open a new notebook to run pandas and gensim 

In [2]:
import pandas as pd
df = pd.read_csv('/Volumes/EXTERNAL1/MIMICIII/select_noteevents2-v3/part-00000-ac3edd68-1571-4e16-a2c6-ada909da0eea.csv',sep='delimiter')
df.head(5)

/Users/cesarosorio/anaconda/envs/BDFH_project/lib/python2.7/site-packages/ipykernel/__main__.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  from ipykernel import kernelapp as app


,text
0,"""Admission Date: [**2183-9-25**] Discha..."
1,"""Admission Date: [**2184-1-16**] Discha..."
2,"""Admission Date: [**2103-4-11**] ..."
3,"""Admission Date: [**2103-10-7**] Discha..."
4,"""\""Admission Date: [**2131-4-2**] ..."


In [2]:
df_ids = pd.read_csv('/Volumes/EXTERNAL1/MIMICIII/select_noteevents2-v4/part-00000-9b6dc461-8bf9-4ae4-b626-2156a12031ab.csv')
df_ids.head(5)

,subject_id,hadm_id
0,20007,188442
1,20007,193793
2,59883,118446
3,17043,157985
4,7019,189488


In [ ]:
# Cleanning the data
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    text = re.sub('[\W]+', ' ', text.lower()) 
    return text

df['text3'] = df['text'].apply(preprocessor)

In [ ]:
# Create tokens
token_review=[]
for i in range(df['text3'].shape[0]):
    review = df['text3'][i]
    token_review.append([i for i in review.split()])

len(token_review)

### Apply Word2Vec

In [26]:
from gensim.models import Word2Vec
#import gensim.models.Word2Vec
from gensim import utils
from time import time

# assumptions: window is 5 words left and right, eliminate words than dont occur in
# more than 10 docs, use 4 workers for a quadcore machine. Size is the size of vector
# negative=5 implies negative sampling and makes doc2vec faster to train
#model = Doc2Vec(sentence, size=100, window=5, workers=4, min_count=5)


import random

size = 100

#instantiate our  model
model_w2v = Word2Vec(min_count=10, window=5, size=size, sample=1e-3, negative=5, workers=4)

#build vocab over all reviews
model_w2v.build_vocab(token_review)

#We pass through the data set multiple times, shuffling the training reviews each time to improve accuracy.
Idx=list(range(len(token_review)))

t0 = time()
for epoch in range(5):
     random.shuffle(Idx)
     perm_sentences = [token_review[i] for i in Idx]
     model_w2v.train(perm_sentences)
     print(epoch)
    
elapsed=time() - t0
print("Time taken for Word2vec training: ", elapsed, "seconds.")

0
1
2
3
4
('Time taken for Word2vec training: ', 766.5733489990234, 'seconds.')


In [3]:
# saves the word2vec model to be used later.
#model_w2v.save('./model_word2vec')

# open a saved word2vec model 
#import gensim
#model_w2v=gensim.models.Word2Vec.load('./model_word2vec')


#model_w2v.wv.save_word2vec_format('./model_word2vec.txt', binary=False)



#### Run  if you are using Glove type format


In [1]:
# Run this cell if you are using Glove type format
def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    f = open(gloveFile,'r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = [float(val) for val in splitLine[1:]]
        model[word] = embedding
    print("Done.",len(model)," words loaded!")
    return model

model_w2v=loadGloveModel("./model_word2vec.txt")

Loading Glove Model
('Done.', 33837, ' words loaded!')


In [2]:
#import data
import pandas as pd
df = pd.read_csv('/Volumes/EXTERNAL1/MIMICIII/select_noteevents2-v3/part-00000-ac3edd68-1571-4e16-a2c6-ada909da0eea.csv',sep='delimiter')

# Cleanning the data
import re

def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    text = re.sub('[\W]+', ' ', text.lower()) 
    text = re.sub(" \d+", " ", text)
    #text = gensim.parsing.preprocessing.remove_stopwords(text)
    return text


df['text3'] = df['text'].apply(preprocessor)

# Create tokens
token_review=[]
for i in range(df['text3'].shape[0]):
    review = df['text3'][i]
    token_review.append([i for i in review.split()])

len(token_review)

/Users/cesarosorio/anaconda/envs/BDFH_project2/lib/python2.7/site-packages/ipykernel/__main__.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  app.launch_new_instance()


59652

In [25]:
import numpy as np  # Make sure that numpy is imported
from nltk.corpus import stopwords

keys_updated = [word for word in model_w2v.keys() if word not in stopwords.words('english')]
index2word_set=set(keys_updated)

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    #index2word_set = set(model.wv.index2word) #activate if using gensim

    # activate if uploaded text version
    #index2word_set=set(keys_updated)
    
    
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


In [26]:
makeFeatureVec(token_review[0], model_w2v, 100)

array([ 0.32124942,  0.42867058, -0.1607722 , -0.255455  ,  0.80822697,
        1.42863403,  0.42433648,  0.45289934,  0.93406838, -0.44348111,
        0.7356697 ,  0.09904312, -0.51461918,  0.09862627, -0.1717864 ,
       -0.49117593,  0.82210046,  0.3637609 , -0.0391038 ,  0.12883505,
       -0.22609152, -0.75877765,  0.43139248,  0.65939814, -0.26636229,
       -0.18819797, -0.11727432, -0.63130124,  0.17328437, -0.11704872,
        0.42489609,  0.17380838, -0.10591966, -0.43874571,  0.71558041,
        0.359048  ,  0.23391739, -0.40168188,  0.41330036, -0.40955439,
       -0.17525836, -0.42868377,  0.21771216, -0.17365117,  0.32529705,
        0.35938327,  0.22340736,  0.424673  ,  0.9110962 ,  0.38621649,
       -0.30929509, -0.84243643, -0.05837796,  0.20928157, -0.75140584,
        0.50740408, -0.16328134, -0.58607459, -0.46503013,  0.106976  ,
       -0.42876902, -0.49975919, -0.4134568 ,  0.17556357, -0.44255696,
        0.04985585, -0.54240325, -0.17266752,  0.95468491, -0.25

In [27]:
def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 10000th review
       if counter%10000 == 0:
           print "Review %d of %d" % (counter, len(reviews))
       # 
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[counter] = makeFeatureVec(review, model,num_features)
       #
       # Increment the counter
       counter = counter + 1
    return reviewFeatureVecs

In [28]:
from time import time
t0 = time()
final_w2v=getAvgFeatureVecs(token_review, model_w2v, num_features=100)
elapsed=time() - t0
print("Time taken for Word2vec avg vector per note calculation: ", elapsed, "seconds.")

Review 0 of 59652
Review 10000 of 59652
Review 20000 of 59652
Review 30000 of 59652
Review 40000 of 59652
Review 50000 of 59652
('Time taken for Word2vec avg vector per note calculation: ', 217.7224440574646, 'seconds.')


#### Create train set and test set to use Machine Learning model


In [29]:
# Create train set and test set to use Machine Learning model
#you need to add the labels
import random
idx=list(range(len(token_review)))
random.seed(1234)
random.shuffle(idx)
idx_train= idx[0:int(len(final_w2v)*0.50)]
idx_val= idx[int(len(final_w2v)*0.50):int(len(final_w2v)*0.75)]
idx_test= idx[int(len(final_w2v)*0.75):len(final_w2v)]

train_set = [final_w2v[i] for i in idx_train]
val_set = [final_w2v[i] for i in idx_val]
test_set = [final_w2v[i] for i in idx_test]


print(len(train_set), len(val_set)  ,len(test_set))

(29826, 14913, 14913)


#### Top 10 notes similar to note '0' or first note in dataset (to compare results to Doc2vec)


In [30]:
# Top 10 notes similar to note '0' or first note in dataset (to compare results to Doc2vec)

import scipy
from scipy import spatial

cosine_results=[]
for i in range(0,final_w2v.shape[0]):
    result=0
    result = 1 - spatial.distance.cosine(final_w2v[0], final_w2v[i])
    cosine_results.append(float("{0:.3f}".format(result)))
    
list_index=sorted(range(len(cosine_results)), key=lambda i: cosine_results[i] , reverse=True)[:11]
for i in list_index:
    print i, cosine_results[i]


0 1.0
10632 0.864
54970 0.863
59628 0.851
58246 0.84
53046 0.833
56235 0.83
53047 0.828
58117 0.827
36272 0.823
1379 0.82


### Try TFIDF

In [31]:
# Transform the dataframe to a sparse Tdidf matrix
#from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

# Now we create the sparse matrix of tfidf values
tfidf = TfidfVectorizer(input='content',ngram_range=(1, 1),stop_words='english', min_df=10, max_df=0.8)
# I select to remove stopwords and minimun doc frequency =10 to delete very unusual words
# that only show up in less than 10 notes (out of 59k notes available) 

dtm = tfidf.fit_transform([c for c in df['text3']])
vocab = np.array(tfidf.get_feature_names())
dtm = dtm.toarray()  # convert to a regular array
vocab = np.array(vocab)
dtm.shape


(59652, 24345)

In [32]:
# Create train set and test set to use Machine Learning model
# you need to add the label
import random
idx=list(range(len(token_review)))
random.seed(1234)
random.shuffle(idx)
idx_train= idx[0:int(len(final_w2v)*0.50)]
idx_val= idx[int(len(final_w2v)*0.50):int(len(final_w2v)*0.75)]
idx_test= idx[int(len(final_w2v)*0.75):len(final_w2v)]

train_set_tfidf = [dtm[i] for i in idx_train]
val_set_tfidf = [dtm[i] for i in idx_val]
test_set_tfidf = [dtm[i] for i in idx_test]

print(len(train_set_tfidf), len(val_set_tfidf)  ,len(test_set_tfidf))

(29826, 14913, 14913)
